In [1]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')


# Data Preprocessing

In [2]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")

symbols_list = sp500[0]['Symbol'].str.replace('.', '-').unique().tolist()

end_date = '2023-09-27'

start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

#Changes the dimensions of the df such that the tickers become an index using .stack()
df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end = end_date).stack()


[*********************100%%**********************]  503 of 503 completed

4 Failed downloads:
['VLTO', 'GEV', 'SOLV', 'SW']: YFChartError("%ticker%: Data doesn't exist for startDate = 1443499200, endDate = 1695787200")


In [3]:
#Changes name of indexes
df.index.names = ['data', 'ticker']

#Changes value of columns to lower case
df.columns = df.columns.str.lower()

In [92]:
# Measures volatility of stock
df['garman_klass_vol'] = (((np.log(df['high'])-np.log(df['low'])))**2)/2 - (2*np.log(2)-1)*(np.log(df['adj close'])-np.log(df['open']))**2

# Create RSI plot for momentum indicator
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

#Create Bbands to see stock volatility
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=x, length=20).iloc[:, 0])
df['bb_avg'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=x, length=20).iloc[:, 1])
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=x, length=20).iloc[:, 2])

# Standardize the Bollinger Bands columns
bb_columns = ['bb_low', 'bb_avg', 'bb_high']

for col in bb_columns:
    df[f'{col}'] = df.groupby(level=1)[col].transform(lambda x: (x - x.mean()) / x.std())

#Calculates normalized ATR price movement of stock
def natr_stock(stock_data):
    atr = pandas_ta.natr(high=stock_data['high'],
            low=stock_data['low'],
            close=stock_data['close'],
            length=14)
    return atr

atr_df = df.groupby(level=1, group_keys=False).apply(natr_stock)

#Calculates normalized MACD of stock
def compute_magc(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:, 0]
    return macd.transform(lambda x: (x-x.mean())/x.std())

df['macd'] = df.groupby(level=1, group_keys=False)['close'].apply(compute_magc)

#Calculate Dollar Volume
df['dollar_volume'] = (df['adj close']*df['volume'])/1e6


In [93]:
df

adj close       close        high         low        open  \
data       ticker                                                               
2015-09-29 A        31.425238   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.651133   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.334900   52.790001   54.189999   51.880001   53.099998   
           ABT      33.478695   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211006  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534828  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.547699  176.869995  178.449997  176.270004  176.580002   

                        volume  garman_klass_vol        rsi    bb_avg  \
data       ticker                                                       
2015-09-29 A         2252400.0         -0.001082        NaN       NaN   
           AAL       7478800.0         -0.000443        NaN       NaN   
           AAPL    293461600.0         -0.005712        NaN       NaN   
           ABBV     12842800.0         -0.054655        NaN       NaN   
           ABT      12287500.0         -0.009402        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0         -0.000018  26.146754  0.825471   
           YUM       1500600.0         -0.000051  36.057175  1.423750   
           ZBH       3610500.0          0.000022  31.893239  0.032680   
           ZBRA       355400.0          0.000133  29.494977  0.125892   
           ZTS       1463200.0          0.000062  42.623467  1.274868   

                    bb_high    bb_low      macd  dollar_volume  
data       ticker                                               
2015-09-29 A            NaN       NaN       NaN      70.782205  
           AAL          NaN       NaN       NaN     279.420098  
           AAPL         NaN       NaN       NaN    7234.160810  
           ABBV         NaN       NaN       NaN     466.641852  
           ABT          NaN       NaN       NaN     411.369464  
...                     ...       ...       ...            ...  
2023-09-26 XYL     0.925470  0.705393 -2.126330     117.344880  
           YUM     1.380047  1.455283 -1.341382     183.389836  
           ZBH    -0.064065  0.130633 -0.850886     402.696497  
           ZBRA    0.224287  0.012713 -1.600791      79.595386  
           ZTS     1.293316  1.248170 -1.171128     256.861393  

[985955 rows x 13 columns]